# 트레인 데이터로만 우선 인포머에 넣고 돌려보기

In [268]:
!pwd

/home/ubuntu/jonghee/gitclone/Informer2020


In [270]:
# libraries
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os
import numpy as np
import time

from sklearn.model_selection import GroupKFold
from sklearn import metrics

sub = pd.read_csv('../../dataset/sample_submission.csv')
train = pd.read_csv('../../dataset/train.csv')
test = pd.read_csv('../../dataset/test.csv')

In [271]:
train

,id,breath_id,R,C,time_step,u_in,u_out,pressure
0,1,1,20,50,0.000000,0.083334,0,5.837492
1,2,1,20,50,0.033652,18.383041,0,5.907794
2,3,1,20,50,0.067514,22.509278,0,7.876254
3,4,1,20,50,0.101542,22.808822,0,11.742872
4,5,1,20,50,0.135756,25.355850,0,12.234987
...,...,...,...,...,...,...,...,...
6035995,6035996,125749,50,10,2.504603,1.489714,1,3.869032
6035996,6035997,125749,50,10,2.537961,1.488497,1,3.869032
6035997,6035998,125749,50,10,2.571408,1.558978,1,3.798729
6035998,6035999,125749,50,10,2.604744,1.272663,1,4.079938


In [272]:
test

,id,breath_id,R,C,time_step,u_in,u_out
0,1,0,5,20,0.000000,0.000000,0
1,2,0,5,20,0.031904,7.515046,0
2,3,0,5,20,0.063827,14.651675,0
3,4,0,5,20,0.095751,21.230610,0
4,5,0,5,20,0.127644,26.320956,0
...,...,...,...,...,...,...,...
4023995,4023996,125748,20,10,2.530117,4.971245,1
4023996,4023997,125748,20,10,2.563853,4.975709,1
4023997,4023998,125748,20,10,2.597475,4.979468,1
4023998,4023999,125748,20,10,2.631134,4.982648,1


## 인포머에 넣을 수 있는 형태로 데이터 가공하기

https://min23th.tistory.com/24

### 날짜 변환

In [273]:
import datetime

In [274]:
train = train.reset_index()
train

,index,id,breath_id,R,C,time_step,u_in,u_out,pressure
0,0,1,1,20,50,0.000000,0.083334,0,5.837492
1,1,2,1,20,50,0.033652,18.383041,0,5.907794
2,2,3,1,20,50,0.067514,22.509278,0,7.876254
3,3,4,1,20,50,0.101542,22.808822,0,11.742872
4,4,5,1,20,50,0.135756,25.355850,0,12.234987
...,...,...,...,...,...,...,...,...,...
6035995,6035995,6035996,125749,50,10,2.504603,1.489714,1,3.869032
6035996,6035996,6035997,125749,50,10,2.537961,1.488497,1,3.869032
6035997,6035997,6035998,125749,50,10,2.571408,1.558978,1,3.798729
6035998,6035998,6035999,125749,50,10,2.604744,1.272663,1,4.079938


In [275]:
# 컬럼 이름 바꾸기
train = train.rename(columns={'index':'date'})
train

,date,id,breath_id,R,C,time_step,u_in,u_out,pressure
0,0,1,1,20,50,0.000000,0.083334,0,5.837492
1,1,2,1,20,50,0.033652,18.383041,0,5.907794
2,2,3,1,20,50,0.067514,22.509278,0,7.876254
3,3,4,1,20,50,0.101542,22.808822,0,11.742872
4,4,5,1,20,50,0.135756,25.355850,0,12.234987
...,...,...,...,...,...,...,...,...,...
6035995,6035995,6035996,125749,50,10,2.504603,1.489714,1,3.869032
6035996,6035996,6035997,125749,50,10,2.537961,1.488497,1,3.869032
6035997,6035997,6035998,125749,50,10,2.571408,1.558978,1,3.798729
6035998,6035998,6035999,125749,50,10,2.604744,1.272663,1,4.079938


In [276]:
def change_index_to_date(value):
    date = datetime.datetime.fromtimestamp(int(value)).strftime('%Y-%m-%d %H:%M:%S')
    return date

train.date = train.date.apply(lambda x: change_index_to_date(x))
train.date

0          1970-01-01 09:00:00
1          1970-01-01 09:00:01
2          1970-01-01 09:00:02
3          1970-01-01 09:00:03
4          1970-01-01 09:00:04
                  ...         
6035995    1970-03-12 05:39:55
6035996    1970-03-12 05:39:56
6035997    1970-03-12 05:39:57
6035998    1970-03-12 05:39:58
6035999    1970-03-12 05:39:59
Name: date, Length: 6036000, dtype: object

In [284]:
train.drop('id', axis=1, inplace=True)

In [285]:
train

,date,breath_id,R,C,time_step,u_in,u_out,pressure
0,1970-01-01 09:00:00,1,20,50,0.000000,0.083334,0,5.837492
1,1970-01-01 09:00:01,1,20,50,0.033652,18.383041,0,5.907794
2,1970-01-01 09:00:02,1,20,50,0.067514,22.509278,0,7.876254
3,1970-01-01 09:00:03,1,20,50,0.101542,22.808822,0,11.742872
4,1970-01-01 09:00:04,1,20,50,0.135756,25.355850,0,12.234987
...,...,...,...,...,...,...,...,...
6035995,1970-03-12 05:39:55,125749,50,10,2.504603,1.489714,1,3.869032
6035996,1970-03-12 05:39:56,125749,50,10,2.537961,1.488497,1,3.869032
6035997,1970-03-12 05:39:57,125749,50,10,2.571408,1.558978,1,3.798729
6035998,1970-03-12 05:39:58,125749,50,10,2.604744,1.272663,1,4.079938


In [360]:
# 모델에 넣기 위해선 csv로 저장을 해놔야한다!
train.to_csv('sample_train.csv', index=False)

In [287]:
pd.read_csv('sample_train.csv')

,date,breath_id,R,C,time_step,u_in,u_out,pressure
0,1970-01-01 09:00:00,1,20,50,0.000000,0.083334,0,5.837492
1,1970-01-01 09:00:01,1,20,50,0.033652,18.383041,0,5.907794
2,1970-01-01 09:00:02,1,20,50,0.067514,22.509278,0,7.876254
3,1970-01-01 09:00:03,1,20,50,0.101542,22.808822,0,11.742872
4,1970-01-01 09:00:04,1,20,50,0.135756,25.355850,0,12.234987
...,...,...,...,...,...,...,...,...
6035995,1970-03-12 05:39:55,125749,50,10,2.504603,1.489714,1,3.869032
6035996,1970-03-12 05:39:56,125749,50,10,2.537961,1.488497,1,3.869032
6035997,1970-03-12 05:39:57,125749,50,10,2.571408,1.558978,1,3.798729
6035998,1970-03-12 05:39:58,125749,50,10,2.604744,1.272663,1,4.079938


## 모델 훈련시키기

https://min23th.tistory.com/24

In [359]:
!pwd

/home/ubuntu/jonghee/gitclone/Informer2020


In [309]:
from utils.tools import dotdict
from exp.exp_informer import Exp_Informer
import torch

In [379]:
args = dotdict()

args.model = 'informer' # model of experiment, options: [informer, informerstack, informerlight(TBD)]

args.data = 'custom' # data
args.root_path = '../../dataset/' # root path of data file
args.data_path = 'sample_train.csv' # data file
args.features = 'MS' # forecasting task, options:[M, S, MS]; M:multivariate predict multivariate, S:univariate predict univariate, MS:multivariate predict univariate
args.target = 'pressure' # target feature in S or MS task
args.freq = 's' # freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h
args.checkpoints = './informer_checkpoints' # location of model checkpoints

args.seq_len = 80 # input sequence length of Informer encoder
args.label_len = 80 # start token length of Informer decoder
args.pred_len = 80 # prediction sequence length
# Informer decoder input: concat[start token series(label_len), zero padding series(pred_len)]

args.enc_in = 7 # encoder input size
args.dec_in = 7 # decoder input size
args.c_out = 1 # output size
args.factor = 5 # probsparse attn factor
args.d_model = 512 # dimension of model
args.n_heads = 8 # num of heads
args.e_layers = 2 # num of encoder layers
args.d_layers = 1 # num of decoder layers
args.d_ff = 2048 # dimension of fcn in model
args.dropout = 0.05 # dropout
args.attn = 'prob' # attention used in encoder, options:[prob, full]
args.embed = 'timeF' # time features encoding, options:[timeF, fixed, learned]
args.activation = 'gelu' # activation
args.distil = True # whether to use distilling in encoder
args.output_attention = False # whether to output attention in ecoder
args.mix = True
args.padding = 0
args.freq = 's' # 오잉 왜 h로 돼있었지??

args.batch_size = 1024 
args.learning_rate = 0.0001
args.loss = 'mae'
args.lradj = 'type1'
args.use_amp = False # whether to use automatic mixed precision training

args.num_workers = 0
args.itr = 1
args.train_epochs = 6
args.patience = 3
args.des = 'exp'

args.use_gpu = True if torch.cuda.is_available() else False
args.gpu = 0

args.use_multi_gpu = True
args.devices = '0,1'


In [296]:
True if torch.cuda.is_available() else False

False

In [290]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())

False
0


In [291]:
torch.__version__

'1.8.0'

In [380]:
args.use_gpu = True if torch.cuda.is_available() and args.use_gpu else False

if args.use_gpu and args.use_multi_gpu:
    args.devices = args.devices.replace(' ','')
    device_ids = args.devices.split(',')
    args.device_ids = [int(id_) for id_ in device_ids]
    args.gpu = args.device_ids[0]

In [381]:
# Set augments by using data name
data_parser = {
    'custom':{'data':'sample_train.csv','T':'pressure','M':[7,7,7],'S':[1,1,1],'MS':[7,7,1]},
}
if args.data in data_parser.keys():
    data_info = data_parser[args.data]
    args.data_path = data_info['data']
    args.target = data_info['T']
    args.enc_in, args.dec_in, args.c_out = data_info[args.features]

In [350]:
args.data

'custom'

In [351]:
data_parser.keys()

dict_keys(['custom'])

In [352]:
if args.data in data_parser.keys():
    print('ok')

ok


In [382]:
data_info
# 오잉 왜 이러는거지...?

{'data': 'sample_train.csv',
 'T': 'pressure',
 'M': [7, 7, 7],
 'S': [1, 1, 1],
 'MS': [7, 7, 1]}

In [383]:
args.detail_freq = args.freq
args.freq = args.freq[-1:]

In [384]:
print('Args in experiment:')
print(args)

Args in experiment:
{'model': 'informer', 'data': 'custom', 'root_path': '../../dataset/', 'data_path': 'sample_train.csv', 'features': 'MS', 'target': 'pressure', 'freq': 's', 'checkpoints': './informer_checkpoints', 'seq_len': 80, 'label_len': 80, 'pred_len': 80, 'enc_in': 7, 'dec_in': 7, 'c_out': 1, 'factor': 5, 'd_model': 512, 'n_heads': 8, 'e_layers': 2, 'd_layers': 1, 'd_ff': 2048, 'dropout': 0.05, 'attn': 'prob', 'embed': 'timeF', 'activation': 'gelu', 'distil': True, 'output_attention': False, 'mix': True, 'padding': 0, 'batch_size': 1024, 'learning_rate': 0.0001, 'loss': 'mae', 'lradj': 'type1', 'use_amp': False, 'num_workers': 0, 'itr': 1, 'train_epochs': 6, 'patience': 3, 'des': 'exp', 'use_gpu': False, 'gpu': 0, 'use_multi_gpu': True, 'devices': '0,1', 'detail_freq': 's'}


In [385]:
Exp = Exp_Informer

In [386]:
range(args.itr)

range(0, 1)

In [256]:
!nvidia-smi

Sat Oct 30 14:19:35 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  On   | 00000000:00:06.0 Off |                    0 |
| N/A   40C    P0    28W / 250W |      0MiB / 32480MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-PCIE...  On   | 00000000:00:07.0 Off |                    0 |
| N/A   42C    P0    27W / 250W |      0MiB / 32480MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

In [ ]:
for ii in range(args.itr):
    # setting record of experiments
    setting = '{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_at{}_fc{}_eb{}_dt{}_mx{}_{}_{}'.format(args.model, args.data, args.features, 
                args.seq_len, args.label_len, args.pred_len,
                args.d_model, args.n_heads, args.e_layers, args.d_layers, args.d_ff, args.attn, args.factor, args.embed, args.distil, args.mix, args.des, ii)

    # set experiments
    exp = Exp(args)
    
    # train
    print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
    exp.train(setting)
    
    # test
    print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
    exp.test(setting)

    torch.cuda.empty_cache()

Use CPU
>>>>>>>start training : informer_custom_ftMS_sl80_ll80_pl80_dm512_nh8_el2_dl1_df2048_atprob_fc5_ebtimeF_dtTrue_mxTrue_exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 4225041
val 603521
test 1207121
	iters: 100, epoch: 1 | loss: 0.4501949
	speed: 24.7412s/iter; left time: 610042.9039s
	iters: 200, epoch: 1 | loss: 0.4140911
	speed: 24.5208s/iter; left time: 602158.0163s
	iters: 300, epoch: 1 | loss: 0.3995913
	speed: 24.6543s/iter; left time: 602971.0799s
	iters: 400, epoch: 1 | loss: 0.4404877
	speed: 24.6541s/iter; left time: 600500.3403s
	iters: 500, epoch: 1 | loss: 0.3937310
	speed: 24.9093s/iter; left time: 604225.6463s
	iters: 600, epoch: 1 | loss: 0.4018248
	speed: 24.7145s/iter; left time: 597028.5281s
	iters: 700, epoch: 1 | loss: 0.4088909
	speed: 24.8623s/iter; left time: 598112.2871s
	iters: 800, epoch: 1 | loss: 0.3782006
	speed: 24.8580s/iter; left time: 595523.2293s
	iters: 900, epoch: 1 | loss: 0.3881542
	speed: 24.7922s/iter; left time: 591466.6955s
	iters: 1000, epoch: 

# 훈련이 된다....

### iters는 1 에폭을 돌기 위해 '전체 데이터의 수/배치사이즈' 이다.

### 즉, 1에폭을 돌기 위해 필요한 배치의 수이다.

https://towardsdatascience.com/epoch-vs-iterations-vs-batch-size-4dfb9c7ce9c9

https://gooopy.tistory.com/68